In [35]:
!pip install azure-storage-blob
!pip install boto3
!pip install google_cloud-storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 38.0 MB/s eta 0:00:0000:0100:01


In [37]:
# import librairies
import pandas as pd
import numpy as np
import json
import requests
import boto3
from azure.storage.blob import BlobServiceClient, ContainerClient
from google.cloud import storage
from io import StringIO

In [22]:
#s1 -> Gather - Deccompress - Convert to Tabular - Clean → Reformat > Consolidation ->Transformation - Load

In [23]:
#import data by using laptop path
file_path = "/Users/lflapple/Desktop/9440/HW/Program_Budget_Operating_Budget_Vs_Expense_Raw_Data_20240417.csv"
df_raw = pd.read_csv(file_path)

print(df_raw.head())




   BUDGET_FISCAL_YEAR  THRU_QUARTER  DEPT_ROLLUP DEPT_ROLLUP_NAME  \
0                2024             2           11    Austin Energy   
1                2024             2           11    Austin Energy   
2                2024             2           11    Austin Energy   
3                2024             2           11    Austin Energy   
4                2024             2           11    Austin Energy   

   DEPARTMENT_CODE DEPARTMENT_NAME FUND_CODE           FUND_NAME PROGRAM_CODE  \
0             1100   Austin Energy      5010  Austin Energy Fund         1PSM   
1             1100   Austin Energy      5010  Austin Energy Fund         1PSM   
2             1100   Austin Energy      5010  Austin Energy Fund         1PSM   
3             1100   Austin Energy      5010  Austin Energy Fund         1PSM   
4             1100   Austin Energy      5010  Austin Energy Fund         1PSM   

                                        PROGRAM_NAME ACTIVITY_CODE  \
0  Power Generation, Market 

In [25]:
import pandas as pd
import requests
from io import StringIO

# API Endpoint
api_endpoint = "https://data.austintexas.gov/resource/g5k8-8sud.csv?$query=SELECT%0A%20%20%60budget_fiscal_year%60%2C%0A%20%20%60thru_quarter%60%2C%0A%20%20%60dept_rollup%60%2C%0A%20%20%60dept_rollup_name%60%2C%0A%20%20%60department_code%60%2C%0A%20%20%60department_name%60%2C%0A%20%20%60fund_code%60%2C%0A%20%20%60fund_name%60%2C%0A%20%20%60program_code%60%2C%0A%20%20%60program_name%60%2C%0A%20%20%60activity_code%60%2C%0A%20%20%60activity_name%60%2C%0A%20%20%60unit_code%60%2C%0A%20%20%60unit_name%60%2C%0A%20%20%60expense_code%60%2C%0A%20%20%60expense_name%60%2C%0A%20%20%60budget%60%2C%0A%20%20%60expenditures%60%2C%0A%20%20%60key%60"

# Send GET request and get response
response = requests.get(api_endpoint)

# Check response status
if response.status_code == 200:
    # Read CSV data from response content
    csv_data = StringIO(response.text)
    
    # Load CSV data into DataFrame
    df = pd.read_csv(csv_data)
    print(df.head())  # Print first few rows of data
else:
    print("Failed to fetch data from API")



   budget_fiscal_year  thru_quarter  dept_rollup dept_rollup_name  \
0                2024             2           11    Austin Energy   
1                2024             2           11    Austin Energy   
2                2024             2           11    Austin Energy   
3                2024             2           11    Austin Energy   
4                2024             2           11    Austin Energy   

   department_code department_name  fund_code           fund_name  \
0             1100   Austin Energy       5010  Austin Energy Fund   
1             1100   Austin Energy       5010  Austin Energy Fund   
2             1100   Austin Energy       5010  Austin Energy Fund   
3             1100   Austin Energy       5010  Austin Energy Fund   
4             1100   Austin Energy       5010  Austin Energy Fund   

  program_code                                       program_name  \
0         1PSM  Power Generation, Market Operations & Resource...   
1         1PSM  Power Generation

In [26]:
print(df_raw.shape)

(56523, 19)


In [28]:
print(df_raw.columns)

Index(['BUDGET_FISCAL_YEAR', 'THRU_QUARTER', 'DEPT_ROLLUP', 'DEPT_ROLLUP_NAME',
       'DEPARTMENT_CODE', 'DEPARTMENT_NAME', 'FUND_CODE', 'FUND_NAME',
       'PROGRAM_CODE', 'PROGRAM_NAME', 'ACTIVITY_CODE', 'ACTIVITY_NAME',
       'UNIT_CODE', 'UNIT_NAME', 'EXPENSE_CODE', 'EXPENSE_NAME', 'BUDGET',
       'EXPENDITURES', 'KEY'],
      dtype='object')


In [30]:
df_raw.dropna()
df_raw.shape

(56523, 19)

In [31]:
df_raw.info()
#no null value

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56523 entries, 0 to 56522
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   BUDGET_FISCAL_YEAR  56523 non-null  int64  
 1   THRU_QUARTER        56523 non-null  int64  
 2   DEPT_ROLLUP         56523 non-null  int64  
 3   DEPT_ROLLUP_NAME    56523 non-null  object 
 4   DEPARTMENT_CODE     56523 non-null  int64  
 5   DEPARTMENT_NAME     56523 non-null  object 
 6   FUND_CODE           56523 non-null  object 
 7   FUND_NAME           56523 non-null  object 
 8   PROGRAM_CODE        56523 non-null  object 
 9   PROGRAM_NAME        56523 non-null  object 
 10  ACTIVITY_CODE       56523 non-null  object 
 11  ACTIVITY_NAME       56523 non-null  object 
 12  UNIT_CODE           56523 non-null  object 
 13  UNIT_NAME           56523 non-null  object 
 14  EXPENSE_CODE        56523 non-null  object 
 15  EXPENSE_NAME        56523 non-null  object 
 16  BUDG

In [47]:
# Function

# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

# Google Cloud Functions
def google_upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

def google_download_blob(bucket_name, source_blob_name, destination_file_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    print(f"Blob {source_blob_name} downloaded to {destination_file_name}.")

# AWS Functions
def aws_upload_file(file_name, bucket, object_name=None):
    if object_name is None:
        object_name = os.path.basename(file_name)
    s3_client = boto3.client('s3')
    response = s3_client.upload_file(file_name, bucket, object_name)
    print(f"Uploaded {file_name} to S3 bucket {bucket}.")

def aws_download_file(bucket, object_name, file_name):
    s3_client = boto3.client('s3')
    s3_client.download_file(bucket, object_name, file_name)
    print(f"Downloaded {object_name} from S3 bucket {bucket}.")

In [49]:
#specify the path to your json configuration file
config_file_path ='config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)
#Print the configuration 
#Connection_STRING = config(["connectionString"])


CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = 'budgetexpense'
blob_name = "budgetexpense_20240418.csv"
# Convert DataFrame to CSV
output = StringIO()
df_raw. to_csv (output, index=False)
data = output. getvalue()
output. close ( )

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get a blob client using the container name and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(data, overwrite=True)
                                                  
print(f"Uploaded (blob_name) to Azure Blob Storage in container (CONTAINER_AZURE).")

Uploaded (blob_name) to Azure Blob Storage in container (CONTAINER_AZURE).
